### 네이버 데이터랩

영화 이름과 기간을 입력해 해당기간에 해당영화를 얼마나 검색했는지 확인

In [1]:
import os
import sys
import json
import urllib.request
from datetime import date
import pandas as pd
from dateutil.relativedelta import relativedelta
from tqdm import tqdm_notebook

df1 = pd.read_csv('/home/ubuntu/movie/data/naver_movie_all.csv')
start_date = date(2017,1,1)
end_date = date(2017,12,31)
len(df1)

0

In [ ]:
m_name = df1['movieNm'][0]
m_code = df1['movieCd'][0]
n_code = df1['naver_movieCd'][0]
m_code

In [ ]:
with open("/home/ubuntu/movie/raw_data/movie_info/"+str(m_code)+".json") as data_file:    
    data = json.load(data_file)
openDt = data['movieInfoResult']['movieInfo']['openDt']
openDt


In [ ]:
date(int(str(openDt)[0:4]),int(str(openDt)[4:6]),int(str(openDt)[6:8]))
edt = date(int(str(openDt)[0:4]),int(str(openDt)[4:6]),int(str(openDt)[6:8])) + relativedelta(months=1)
sdt = date(int(str(openDt)[0:4]),int(str(openDt)[4:6]),int(str(openDt)[6:8]))
str(sdt)

In [ ]:
type(str(n_code))

In [ ]:
for i in tqdm_notebook(range(len(df1['naver_movieCd']))):
    m_name = df1['movieNm'][i]
    m_code = df1['movieCd'][i]
    n_code = df1['naver_movieCd'][i]
    
    with open('/home/ubuntu/movie/raw_data/movie_info/'+str(m_code)+'.json') as data_file:    
        data = json.load(data_file)
    openDt = data['movieInfoResult']['movieInfo']['openDt']
    
    idt = date(2016,1,1)
    sdt = date(int(str(openDt)[0:4]),int(str(openDt)[4:6]),int(str(openDt)[6:8]))
    edt = sdt + relativedelta(months=1)
    
    if(sdt > idt):
        client_id = "nA8Iwm4HJ6EM2W1dueU9"
        client_secret = "HaEPQCPbyQ"
        url = "https://openapi.naver.com/v1/datalab/search"

        # body='{\"startDate\":\"'+str(sdt)+'\",\"endDate\":\"2017-01-21\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\"영화\",\"keywords\":[\"'+'꾼'+'\"]}]}'
        body='{\"startDate\":\"'+str(sdt)+'\",\"endDate\":\"'+str(edt)+'\",\"timeUnit\":\"date\",\"keywordGroups\":[{\"groupName\":\"영화\",\"keywords\":[\"'+m_name+'\"]}]}'

        print(body)
        request = urllib.request.Request(url)
        request.add_header("X-Naver-Client-Id",client_id)
        request.add_header("X-Naver-Client-Secret",client_secret)
        request.add_header("Content-Type","application/json")
        response = urllib.request.urlopen(request, data=body.encode("utf-8"))
        rescode = response.getcode()
        if(rescode==200):
            response_body = response.read()

            d = json.loads(response_body)
        #     print(response_body.decode('utf-8'))
#             with open('data2/datalab/'+str(n_code)+'.json', 'w', encoding="utf-8") as make_file:
#                 json.dump(d, make_file, ensure_ascii=False, indent="\t")
        else:
            print("Error Code:" + rescode)
            
        d['results'][0]['data']
        df = pd.DataFrame(d['results'][0]['data'])
        df.to_csv('/home/ubuntu/movie/data/datalab/'+str(n_code)[:-2]+'_datalab.csv',index=False)
    else:
        pass

In [ ]:
from tqdm import tqdm_notebook
df1 = pd.read_csv('/home/ubuntu/movie/data/naver_movie_all.csv')

for i in tqdm_notebook(range(len(df1))): #range(len(df1)):
    ncd = str(df1['naver_movieCd'][i])[:-2]
#     print(ncd)
    try:
        df2 = pd.read_csv('/home/ubuntu/movie/data/datalab/'+ncd+'_datalab.csv')
        df3 = pd.read_csv('/home/ubuntu/movie/data/after_rating/'+ncd+'_rating.csv')
    
        for i in range(len(df3)):
            df3['0'][i] = str(df3['0'][i])[0:4]+"-"+str(df3['0'][i])[4:6]+"-"+str(df3['0'][i])[6:8]
        df3.rename(columns={"0":"period"}, inplace = True)
        df4 = pd.merge(df2,df3,how='left',on='period')
        for i in range(len(df4)):
            df4['1'][i] = df4['1'][i] * 100
            df4['2'][i] = df4['2'][i] * 10
        df4.to_csv('/home/ubuntu/movie/data/merge/'+ncd+'_merge.csv',index=False)
    except FileNotFoundError:
        pass